In [1]:
import GCRCatalogs
import matplotlib.pyplot as plt
import pandas as pd
import glob
import numpy as np
from astropy import table
import seaborn as sns

In [2]:
cosmo = GCRCatalogs.load_catalog("cosmoDC2_v1.1.4_image")

In [8]:
def collect_snana_outputs(path): #path may be single or list
    if type(path)==str:
        headfiles = sorted(glob.glob(path+ '*HEAD.FITS*'))
        photfiles = sorted(glob.glob(path+ '*PHOT.FITS*'))
    elif type(path)==list:
        headfiles = []
        photfiles = []
        for p in path:
            headfiles.extend(sorted(glob.glob(p+ '*HEAD.FITS*')))
            photfiles.extend(sorted(glob.glob(p+ '*HEAD.FITS*')))
    c=0
    for file in headfiles: # don't read more than 10
        if c==0:
            headfile = table.Table.read(file, format='fits').to_pandas()
        else:
            headfile=headfile.append(table.Table.read(file, format='fits').to_pandas())
        c+=1
    return headfile
        
def pull_cosmodc2_props(quantities, indices):
    filters=[(lambda x: np.isin(x, indices), 'galaxy_id')]
    quants = quantities+['galaxy_id']
    gals = cosmo.get_quantities(quants, filters)
    return gals

In [9]:
path = "/global/cscratch1/sd/kessler/SNANA_LSST_SIM/SCOTCH10k_Z3/"
snia = collect_snana_outputs(path+"MLAG_SCOTCH10k_SNIa-SALT2/")


In [10]:
sniax = collect_snana_outputs(path+"MLAG_SCOTCH10k_SNIax/")


In [11]:
sn91bg = collect_snana_outputs(path+"MLAG_SCOTCH10k_SNIa-91bg/")

In [12]:
# snia   = snia.sample(n=10000)
# sniax  = sniax.sample(n=10000)
# sn91bg = sn91bg.sample(n=10000)

In [13]:
quantities= ['Mag_true_u_sdss_z0','Mag_true_r_sdss_z0']
snia = pull_cosmodc2_props(quantities, snia['HOSTGAL_OBJID'])


In [14]:
sniax = pull_cosmodc2_props(quantities, sniax['HOSTGAL_OBJID'])

In [15]:
sn91bg =  pull_cosmodc2_props(quantities, sn91bg['HOSTGAL_OBJID'])

In [16]:
np.average(snia['Mag_true_u_sdss_z0']-snia['Mag_true_r_sdss_z0']), np.std(snia['Mag_true_u_sdss_z0']-snia['Mag_true_r_sdss_z0'])/np.sqrt(len(snia['Mag_true_r_sdss_z0']))


(1.3723375, 0.005683095448783962)

In [17]:
np.average(sniax['Mag_true_u_sdss_z0']-sniax['Mag_true_r_sdss_z0']), np.std(sniax['Mag_true_u_sdss_z0']-sniax['Mag_true_r_sdss_z0'])/np.sqrt(len(sniax['Mag_true_u_sdss_z0']))

(1.2553247, 0.005814034636491799)

In [18]:
np.average(sn91bg['Mag_true_u_sdss_z0']-sn91bg['Mag_true_r_sdss_z0']), np.std(sn91bg['Mag_true_u_sdss_z0']-sn91bg['Mag_true_r_sdss_z0'])/np.sqrt(len(sn91bg['Mag_true_u_sdss_z0']))


(1.9086872, 0.003283362515673)

In [2]:
# hakobyan 91bg vs normal
print(2.23-1.86)
# ours
print(1.9086872-1.3723375)

0.3699999999999999
0.5363496999999999


In [8]:
# hakobyan Iax vs normal
print(1.47-1.86)
# ours
print(1.2553247-1.3723375)

-0.3900000000000001
-0.11701279999999992


In [6]:
import numpy as np

In [9]:
np.sqrt(0.005683095448783962**2+0.005814034636491799**2)

0.0081302258661316

In [ ]:
np.sqrt(0.005683095448783962**2+0.003283362515673**2)

In [11]:
(1.86-1.3723375)/np.sqrt(.03**2+.005**2)

16.034244229359764

In [12]:
(1.47-1.2553247)/np.sqrt(.1**2+.006**2)

2.1428992466181955

In [13]:
(2.23-1.91)/np.sqrt(.05**2+.003**2)

6.388511010980984